## 📚 Prerequisites

Ensure that your Azure Services are properly set up, your Conda environment is created, and your environment variables are configured as per the instructions in the [README.md](README.md) file.

In [46]:
import os

# Define the target directory
target_directory = r"C:\Users\pablosal\Desktop\gbb-ai-hls-factory-prior-auth"  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbb-ai-hls-factory-prior-auth


In [1]:
from src.aoai.azure_openai import AzureOpenAIManager
from utils.ml_logging import get_logger

# Set up logger
logger = get_logger()

## Rewrite Query 

In [2]:
result_pos = {'Patient Information': {'Patient Name': 'Sarah Sample',
  'Patient Date of Birth': '10-19-2014',
  'Patient ID': '4567890',
  'Patient Address': '25 W Randolph St, Chicago, IL 60601',
  'Patient Phone Number': '555-123-4567'},
 'Physician Information': {'Physician Name': 'Shiva Pedram, MD',
  'Specialty': 'Pediatric Gastroenterology',
  'Physician Contact': {'Office Phone': '555-991-2750',
   'Fax': '555-786-5643',
   'Office Address': '5721 S Maryland Ave, Chicago, IL 60637'}},
 'Clinical Information': {'Diagnosis and medical justification (including ICD-10 code)': "Crohn's Disease (ICD-10 code: K50.90)",
  'Detailed history of alternative treatments and results': 'Methylprednisolone (10/10/24 - 10/19/24): Patient continued to have bloody stools and abdominal pain with no improvement.',
  'Relevant lab results or diagnostic imaging': 'EGD: Gastritis, erythema present, Mild to moderate duodenitis with edema. Colonoscopy: Patchy inflammation with areas of erythema and ulceration, segmental strictures observed, diffuse inflammation with cobblestone appearance. MRI enterography: pending. CBC: Hemoglobin 9.0 g/dL, Hematocrit 32%, RBC 3.5 million/µL, WBC 12,000 cells/µL, Platelets 450,000 cells/µL.',
  'Documented symptom severity and impact on daily life': 'The patient experiences severe abdominal pain, fatigue, pallor, dizziness, and frequent bloody stools, significantly impacting daily life activities and sleep.',
  'Prognosis and risk if treatment is not approved': "If treatment is not approved, the patient is at risk of ongoing severe symptoms, potential complications from Crohn's disease, and continued anemia due to blood loss.",
  'Clinical rationale for urgency (if applicable)': 'The patient continues to have hematochezia and severe symptoms despite current steroid therapy, necessitating urgent biologic therapy initiation.',
  'Plan for Treatment or Request for Prior Authorization': {'Medication or Procedure': 'Adalimumab 40mg',
   'Code': 'J0135',
   'Dosage': '160 mg (given as four 40 mg injections on day 1) followed by 80 mg (given as two 40 mg injections) two weeks later. 40 mg every other week starting 2 weeks from 2nd dose.',
   'Duration': '6 months',
   'Rationale': "Given the severity of the patient's symptoms and lack of improvement with steroid therapy, Adalimumab is necessary to manage Crohn's disease effectively and prevent further complications."}}}


In [3]:
results_neg = {'Patient Information': {'Patient Name': 'Sarah Sample',
  'Patient Date of Birth': '10-19-2014',
  'Patient ID': '4567890',
  'Patient Address': '28 W Comet Ave, Chicago, IL 60607',
  'Patient Phone Number': '555-123-4567'},
 'Physician Information': {'Physician Name': 'Shiva Pedram, MD',
  'Specialty': 'Pediatrics',
  'Physician Contact': {'Office Phone': '555-991-2750',
   'Fax': '555-786-5643',
   'Office Address': '5721 S Maryland Ave, Chicago, IL 60637'}},
 'Clinical Information': {'Diagnosis and medical justification (including ICD-10 code)': 'Crohn’s Disease, ICD-10: K50.90',
  'Detailed history of alternative treatments and results': 'Not provided',
  'Relevant lab results or diagnostic imaging': 'EGD: Stomach - Gastritis, erythema present; Duodenum - Mild to moderate duodenitis with edema. Colonoscopy: Ileum - Patchy inflammation with areas of erythema and ulceration; Colon - Diffuse inflammation, granularity, and friability present; Rectum - Mild inflammation, no significant ulceration. CBC with differential: Hemoglobin 9.0 g/dL (11.5-15.5 g/dL), Hematocrit 32% (34.5-44.5%), Red Blood Cells 3.5 million/μL (4.0-5.5 million/μL), Platelets 450,000 cells/μL (150,000-400,000 cells/μL), WBC 12,000 cells/μL (4,500-13,500 cells/μL). CMP: Glucose 90 mg/dL (70-100 mg/dL), BUN 15 mg/dL (7-20 mg/dL), Creatinine 0.5 mg/dL (0.3-0.7 mg/dL). Other tests: ESR 15 mm/h (<20 mm/h), CRP 12 mg/L (<5 mg/L), Fecal Calprotectin 100 μg/g (<50 μg/g), Fecal Occult Blood - Positive.',
  'Documented symptom severity and impact on daily life': 'Patient has multiple episodes of abdominal cramping and bowel movements with visible hematochezia, visible fatigue, pallor, and dizziness. Continued blood loss and anemia.',
  'Prognosis and risk if treatment is not approved': 'Continued frequent blood in stools and anemia as well as abdominal discomfort, risking long-term gastrointestinal damage and deterioration in quality of life.',
  'Clinical rationale for urgency (if applicable)': 'Patient symptomatic with pallor, dizziness, blood loss. Urgent need for biologic therapy to manage severe symptoms and prevent further complications.',
  'Plan for Treatment or Request for Prior Authorization': {'Medication or Procedure': 'Adalimumab',
   'Code': 'Not provided',
   'Dosage': '40mg',
   'Duration': '6 months',
   'Rationale': 'Biologic therapy indicated given severity of symptoms and poor response to alternative treatments.'}}}

## Proccess Images 

In [4]:
azure_openai_client_05_13 = AzureOpenAIManager(completion_model_name='AZURE_OPENAI_CHAT_DEPLOYMENT_ID')

In [5]:
SYSTEM_PROMPT = """
## Role:
You are an expert in search engine evaluation and prior authorization specialist.

## Task:
Your task is to review the clinical evaluation and documentation provided in JSON format and return a query that will maximize the likelihood of finding the exact matching policy.

## Instructions:
Carefully analyze the provided clinical information, including:
1. Diagnosis and medical justification (including ICD-10 code)
2. Detailed history of alternative treatments and results
3. Relevant lab results or diagnostic imaging
4. Documented symptom severity and impact on daily life
5. Prognosis and risks if treatment is not approved
6. Clinical rationale for urgency (if applicable)
7. Plan for Treatment or Request for Prior Authorization:
  - Name of the medication or procedure being requested
  - Code of the medication or procedure (if available)
  - Dosage or plan for the medication or procedure
  - Duration of the treatment
  - Rationale for the medication or procedure

Your goal is to construct a search query that uses this clinical information to retrieve the most relevant prior authorization policy. Leverage vector and embedding-based techniques to enhance search accuracy and ensure the query reflects the urgency and specific medical needs of the case.
"""

In [6]:
USER_PROMPT_1 = f'''
## Role:
You are an expert in search engine evaluation and prior authorization.

## Task:
Your task is to review the following clinical information and construct a search query that will maximize the likelihood of finding the exact matching prior authorization policy in Azure AI Search.

## Instructions:
Carefully analyze the provided clinical information:

- **Diagnosis and Medical Justification** (including ICD-10 code)
- **Detailed History of Alternative Treatments and Their Results**
- **Relevant Lab Results or Diagnostic Imaging**
- **Documented Symptom Severity and Its Impact on Daily Life**
- **Prognosis and Risks if Treatment is Not Approved**
- **Clinical Rationale for Urgency** (if applicable)
- **Plan for Treatment or Request for Prior Authorization**:
  - Name of the medication or procedure being requested
  - Code of the medication or procedure (if available)
  - Dosage or plan for the medication or procedure
  - Duration of the treatment
  - Rationale for the medication or procedure

### Provided Clinical Information:
{results_neg.get("Clinical Information", "Not provided")}

## Goal:
Construct a search query that leverages this clinical information to retrieve the most relevant prior authorization policy from Azure AI Search. Utilize vector embeddings and semantic search capabilities to enhance accuracy. Include appropriate keywords and phrases, and ensure the query reflects the urgency and specific medical needs of the case.

### Key Points:
1. **Accuracy**: Ensure the query accurately reflects the provided clinical information.
2. **Keywords and Phrases**: Include relevant keywords and phrases to improve search results.
3. **Urgency and Specific Needs**: Reflect the urgency and specific medical needs in the query.
4. **Vector Embeddings and Semantic Search**: Utilize these capabilities to enhance the accuracy and relevance of the search results.

## Interpreting the Information for Optimal Search:
To maximize the effectiveness of your search:

### Include Specific Medical Terms and Codes:
- "Crohn's Disease," "K50.90," "biologic therapy," "methylprednisolone," "steroid-resistant"

### Highlight Symptom Severity and Treatment History:
- "Severe abdominal pain," "bloody stools," "no improvement with steroids," "urgent treatment needed"

### Emphasize Lab Results and Imaging Findings:
- "Low hemoglobin," "elevated CRP," "cobblestone appearance in colonoscopy," "gastritis and duodenitis"

### Stress Urgency and Risks:
- "Risk of complications," "worsening symptoms," "impact on quality of life"

### Include Keywords for Condition and Medication:
- Ensure to include keywords related to the specific condition and the medication or procedure being sought in the clinical context.

Generate a JSON output based on the following schema and instructions:

Schema:
{{
  "optimized_query": ""  // Please provide the constructed search query.
}}

Please provide the constructed search query.
'''

In [7]:
api_response_gpt4o = await azure_openai_client_05_13.generate_chat_response(query=USER_PROMPT_1, 
                                                           system_message_content=SYSTEM_PROMPT, 
                                                           conversation_history=[],
                                                           stream=False,
                                                           response_format='json_object',
                                                           max_tokens=2000)

2024-10-25 08:40:53,746 - micro - MainProcess - INFO     Function generate_chat_response started at 2024-10-25 08:40:53 (azure_openai.py:generate_chat_response:345)
2024-10-25 08:40:53,747 - micro - MainProcess - INFO     Sending request to Azure OpenAI at 2024-10-25 08:40:53 (azure_openai.py:generate_chat_response:398)
2024-10-25 08:40:55,105 - micro - MainProcess - INFO     Function generate_chat_response finished at 2024-10-25 08:40:55 (Duration: 1.36 seconds) (azure_openai.py:generate_chat_response:444)


In [8]:
api_response_gpt4o['response']['optimized_query']

"Crohn's Disease ICD-10 K50.90 biologic therapy Adalimumab 40mg severe abdominal cramping hematochezia anemia pallor dizziness urgent treatment needed endoscopy colonoscopy gastritis duodenitis inflammation ulceration CBC hemoglobin low CRP elevated fecal calprotectin positive blood loss quality of life deterioration risk of complications 6 months"

In [9]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential

In [10]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential

search_client = SearchClient(
    endpoint=os.environ["AZURE_AI_SEARCH_SERVICE_ENDPOINT"],
    index_name=os.environ["AZURE_AI_SEARCH_INDEX_NAME"],
    credential=AzureKeyCredential(os.environ["AZURE_AI_SEARCH_ADMIN_KEY"]),
)

vector_query = VectorizableTextQuery(text=api_response_gpt4o['response']['optimized_query'],
                                    k_nearest_neighbors=5, 
                                    fields="vector",
                                    weight=0.5)

from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType

results = search_client.search(  
    search_text=api_response_gpt4o['response']['optimized_query'],  
    vector_queries=[vector_query],
    #select=["content", "id"],
    #filters = 
    query_type=QueryType.SEMANTIC, semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
    top=5
)

# Assuming 'results' is an iterable (e.g., SearchItemPaged)
try:
    first_result = next(iter(results))
    parent_path = first_result.get('parent_path', 'Path not found')
    print(parent_path)
except StopIteration:
    print('No results found')

https://testaccountfactoryeastus.blob.core.windows.net/pre-auth-policies/policies_ocr/001_inflammatory_Conditions.pdf


In [11]:
vector_query = VectorizableTextQuery(text=api_response_gpt4o['response']['optimized_query'],
                                    k_nearest_neighbors=5, 
                                    fields="vector",
                                    weight=0.5)

In [12]:
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType

results = search_client.search(  
    search_text=api_response_gpt4o['response']['optimized_query'],  
    vector_queries=[vector_query],
    #select=["content", "id"],
    #filters = 
    query_type=QueryType.SEMANTIC, semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
    top=5
)

# for result in results:
#     print("=" * 40)
#     print(f"ID: {result['chunk_id']}")
#     print(f"Reranker Score: {result['@search.reranker_score']}")
#     print(f"Source_doc_path: {result['parent_path']}")
#     content = result['chunk'][:100] + '...' if len(result['chunk']) > 100 else result['chunk']
#     print(f"Content: {content}")

#     captions = result.get("@search.captions", [])
#     if captions:
#         caption = captions[0]
#         if caption.highlights:
#             print(f"Caption: {caption.highlights}")
#         else:
#             print(f"Caption: {caption.text}")
#     print("=" * 40)


In [13]:
# Assuming 'results' is an iterable (e.g., SearchItemPaged)
try:
    first_result = next(iter(results))
    parent_path = first_result.get('parent_path', 'Path not found')
    print(parent_path)
except StopIteration:
    print('No results found')

https://testaccountfactoryeastus.blob.core.windows.net/pre-auth-policies/policies_ocr/001_inflammatory_Conditions.pdf


In [14]:
from src.ocr.document_intelligence import AzureDocumentIntelligenceManager
document_intelligence_client = AzureDocumentIntelligenceManager()

In [15]:
import os
import urllib.parse
from azure.storage.blob import BlobServiceClient

# Blob URL
blob_url = "https://storagefactoryeastus.blob.core.windows.net/pre-auth-policies/policies_ocr/001_inflammatory_Conditions.pdf"

# Parse the URL to extract the blob name
parsed_url = urllib.parse.urlparse(blob_url)
blob_name = '/'.join(parsed_url.path.split('/')[2:])  # Extract the blob name correctly

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(os.environ["BLOB_CONNECTION_STRING"])

# Extract the container name from the URL
container_name = parsed_url.path.split('/')[1]

# Log the container name and blob name for debugging
print(f"Container Name: {container_name}")
print(f"Blob Name: {blob_name}")

# Get the blob client
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Download the blob content
try:
    blob_content = blob_client.download_blob().readall()
    print("Blob content downloaded successfully.")
except Exception as e:
    print(f"Failed to download blob content: {e}")

# Use the blob content with the document intelligence client
model_type = "prebuilt-layout"

policy_text = document_intelligence_client.analyze_document(
    document_input=blob_content,
    model_type=model_type,
    output_format="markdown",
    features=["OCR_HIGH_RESOLUTION"],
    # pages="1-4",
)

Container Name: pre-auth-policies
Blob Name: policies_ocr/001_inflammatory_Conditions.pdf
Blob content downloaded successfully.


In [16]:
policy_text.content

'<figure>\n\n![](figures/0)\n\n<!-- FigureContent="cigna healthcare" -->\n\n</figure>\n\n\nPRIOR AUTHORIZATION POLICY\n===\n\nPOLICY:\n\nInflammatory Conditions - Adalimumab Products Prior Authorization\n\nPolicy\n\n· Abrilada™ (adalimumab-afzb subcutaneous injection - Pfizer)\n\n· adalimumab-aacf subcutaneous injection (Fresenius Kabi)\n\n· adalimumab-adaz subcutaneous injection (Sandoz/Novartis)\n\n· adalimumab-adbm subcutaneous injection (Boehringer Ingelheim)\n\n· adalimumab-fkjp subcutaneous injection (Mylan)\n\n· adalimumab-ryvk subcutaneous injection (Teva/Alvotech)\n\n· Amjevita® (adalimumab-atto subcutaneous injection - Amgen)\n\n. Cyltezo® (adalimumab-adbm subcutaneous injection - Boehringer Ingelheim)\n\n· Hadlima TM (adalimumab-bwwd subcutaneous injection –\n\nOrganon/Samsung Bioepis)\n\n· Hulio® (adalimumab-fkjp subcutaneous injection - Mylan)\n\n· Humira® (adalimumab subcutaneous injection - AbbVie, Cordavis)\n\n· Hyrimoz® (adalimumab-adaz subcutaneous injection Sandoz/No

In [9]:
# azure_openai_client_01 = AzureOpenAIManager(api_key=os.environ["AZURE_OPENAI_KEY_01"],
#                                             api_version=os.environ["AZURE_OPENAI_API_VERSION_01"],
#                                             chat_model_name=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_01"],
#                                             azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT_01"])

In [62]:
SYSTEM_PROMPT_PRIOR_AUTH = """
## Role:
You are an expert Prior Authorization (PA) specialist with extensive experience in analyzing medical documents and making informed decisions regarding prior authorization requests.

## Task:
Your task is to review the provided policy text extracted from OCR, the clinical information, and the patient information to decide if the prior authorization request should be approved, denied, or if more information is needed.

## Instructions:
Carefully analyze the provided policy text, clinical information, and patient information:

1. **Policy Text**: The text extracted from the policy document using OCR.
2. **Patient Information**:
   - Patient Name
   - Patient Date of Birth
   - Patient ID
   - Patient Address
   - Patient Phone Number
3. **Clinical Information**:
   - Diagnosis and Medical Justification (including ICD-10 code)
   - Detailed History of Alternative Treatments and Their Results
   - Relevant Lab Results or Diagnostic Imaging
   - Documented Symptom Severity and Its Impact on Daily Life
   - Prognosis and Risks if Treatment is Not Approved
   - Clinical Rationale for Urgency (if applicable)
   - Plan for Treatment or Request for Prior Authorization:
     - Name of the medication or procedure being requested
     - Code of the medication or procedure (if available)
     - Dosage or plan for the medication or procedure
     - Duration of the treatment
     - Rationale for the medication or procedure

## Goal:
Based on the provided policy text, patient information, and clinical information, decide if the prior authorization request should be:
1. **Approved**: If the request meets the policy criteria.
2. **Denied**: If the request does not meet the policy criteria.
3. **Needs More Information**: If additional information is required to make a decision.

## Decision Criteria:
1. **Approval**: The request meets all the criteria outlined in the policy text.
2. **Denial**: The request does not meet the criteria outlined in the policy text.
3. **Needs More Information**: The provided information is insufficient to make a decision, and additional details are required.

## Example Decision:
- **Approved**: The request for Adalimumab (J0135) for a patient with Crohn's Disease (ICD-10 K50.90) is approved based on the policy criteria.
- **Denied**: The request for Adalimumab (J0135) for a patient with Crohn's Disease (ICD-10 K50.90) is denied because it does not meet the policy criteria.
- **Needs More Information**: Additional information is required to make a decision on the request for Adalimumab (J0135) for a patient with Crohn's Disease (ICD-10 K50.90).

Please provide your decision along with a brief explanation.
"""

In [63]:
USER_PROMPT_PRIOR_AUTH = f"""
## Patient Information:
- **Patient Name**: {results_neg.get("Patient Information", {}).get("Patient Name", "Not provided")}
- **Patient Date of Birth**: {results_neg.get("Patient Information", {}).get("Patient Date of Birth", "Not provided")}
- **Patient ID**: {results_neg.get("Patient Information", {}).get("Patient ID", "Not provided")}
- **Patient Address**: {results_neg.get("Patient Information", {}).get("Patient Address", "Not provided")}
- **Patient Phone Number**: {results_neg.get("Patient Information", {}).get("Patient Phone Number", "Not provided")}

## Clinical Information:
- **Diagnosis and Medical Justification**: {results_neg.get("Clinical Information", {}).get("Diagnosis and medical justification (including ICD-10 code)", "Not provided")}
- **Detailed History of Alternative Treatments and Their Results**: {results_neg.get("Clinical Information", {}).get("Detailed history of alternative treatments and results", "Not provided")}
- **Relevant Lab Results or Diagnostic Imaging**: {results_neg.get("Clinical Information", {}).get("Relevant lab results or diagnostic imaging", "Not provided")}
- **Documented Symptom Severity and Its Impact on Daily Life**: {results_neg.get("Clinical Information", {}).get("Documented symptom severity and impact on daily life", "Not provided")}
- **Prognosis and Risks if Treatment is Not Approved**: {results_neg.get("Clinical Information", {}).get("Prognosis and risk if treatment is not approved", "Not provided")}
- **Clinical Rationale for Urgency**: {results_neg.get("Clinical Information", {}).get("Clinical rationale for urgency (if applicable)", "Not provided")}
- **Plan for Treatment or Request for Prior Authorization**:
  - **Medication or Procedure**: {results_neg.get("Clinical Information", {}).get("Plan for Treatment or Request for Prior Authorization", {}).get("Medication or Procedure", "Not provided")}
  - **Code**: {results_neg.get("Clinical Information", {}).get("Plan for Treatment or Request for Prior Authorization", {}).get("Code", "Not provided")}
  - **Dosage or Plan**: {results_neg.get("Clinical Information", {}).get("Plan for Treatment or Request for Prior Authorization", {}).get("Dosage", "Not provided")}
  - **Duration**: {results_neg.get("Clinical Information", {}).get("Plan for Treatment or Request for Prior Authorization", {}).get("Duration", "Not provided")}
  - **Rationale**: {results_neg.get("Clinical Information", {}).get("Plan for Treatment or Request for Prior Authorization", {}).get("Rationale", "Not provided")}

## Policy Text:
{policy_text.content}

## Decision:
Based on the provided policy text, patient information, and clinical information, decide if the prior authorization request should be:
1. **Approved**: If the request meets the policy criteria.
2. **Denied**: If the request does not meet the policy criteria.
3. **Needs More Information**: If additional information is required to make a decision.

Please provide your decision along with a brief explanation.
"""

In [64]:
api_response_gpt4o = await azure_openai_client_05_13.generate_chat_response(query=USER_PROMPT_PRIOR_AUTH, 
                                                           system_message_content=SYSTEM_PROMPT_PRIOR_AUTH, 
                                                           conversation_history=[],
                                                           stream=False,
                                                           response_format='text',
                                                           max_tokens=2000)

2024-10-21 22:31:18,196 - micro - MainProcess - INFO     Function generate_chat_response started at 2024-10-21 22:31:18 (azure_openai.py:generate_chat_response:345)
2024-10-21 22:31:18,200 - micro - MainProcess - INFO     Sending request to Azure OpenAI at 2024-10-21 22:31:18 (azure_openai.py:generate_chat_response:398)
2024-10-21 22:31:23,153 - micro - MainProcess - INFO     Function generate_chat_response finished at 2024-10-21 22:31:23 (Duration: 4.96 seconds) (azure_openai.py:generate_chat_response:444)


In [65]:
print(api_response_gpt4o['response'])

## Decision: Needs More Information

### Explanation:
While the provided clinical information indicates that the patient, Sarah Sample, has a diagnosis of Crohn's Disease (ICD-10: K50.90) and presents with severe symptoms that could benefit from Adalimumab therapy, the policy criteria for prior authorization under Cigna's guidelines require additional specific details:

1. **Detailed History of Alternative Treatments and Their Results**:
   - The policy stipulates that for initial therapy approval, the patient must have tried or be currently taking corticosteroids, or corticosteroids must be contraindicated. Alternatively, the patient must have tried one other conventional systemic therapy for Crohn's disease or have specific types of fistulas or have had ileocolonic resection.
   - The provided clinical information does not detail the patient's history of corticosteroid use or other conventional systemic therapies (e.g., azathioprine, 6-mercaptopurine, methotrexate).

2. **Prescribing